In [4]:
from imports import *
from exactDiagMethods import *
from globalMethods import  *
from vqteMethods import *
from tests import *

## H imag * iH real
## Compare matricies in seperate file
## Look at Joaquins python file for this 

In [5]:
def main():
    # Initialize the simulation parameters
    N = 2
    gamma_L= 0.5
    gamma_R = 1.0
    eps = 1
    mu_L= 0.25
    mu_R= 1
    T_L = 10   
    T_R= 1
    beta_L=1/T_L
    beta_R=1/T_R
    F_L = 1/(np.exp(beta_L*(eps-mu_L))+1)
    F_R = 1/(np.exp(beta_R*(eps-mu_R))+1)
    J = 1
    
    #Layer = 2

    eps_list = []

    for i in range(N):
        eps_list.append(1)

  
    time = 6.0
    dt = 0.1
    nt = int(time/dt)

    

    exact_diag_ham = build_exact_diag_hamiltonian(J, eps_list)

    #print("Exact Ham", exact_diag_ham)

    ham_real, ham_imag = hamiltonian_generation(N, eps_list, gamma_L, gamma_R, F_L, F_R, J)

    
    
    # Generate initial states
    vqte_init_state, exact_diag_init_state, ansatz, init_param_values = build_initial_states(ham_real)


    exact_diag_results, superoperator = perform_exact_diag(gamma_L,F_L, gamma_R, F_R, dt, nt, exact_diag_init_state,exact_diag_ham, N)
    
    difference = compare_superoperator_to_vqte(superoperator, ham_real, ham_imag)
    
    print("This is the difference." , difference)

    #print("This is important", superoperator - (ham_imag.to_matrix()+ 1j*ham_real.to_matrix()))
    
 

    
    #print("Superoperator from exact diag method: ", superoperator)
    vqte_results, numoplist = perform_vqte(ham_real, ham_imag, vqte_init_state, dt, nt, ansatz, init_param_values)
    
    #print("Difference", compare_superoperator_to_vqte(superoperator, ham_real, ham_imag) )
    
    #print("Number Operators: ", numoplist)
    # # Output results
    
    output_results(vqte_results, exact_diag_results, time, nt)

In [ ]:
main()

This is the difference. [[-0.74062939-0.74062939j  0.        +0.j          0.        +0.j
   0.        +0.j          0.        +0.j          0.5       +0.5j
   0.        +0.j          0.        +0.j          0.        +0.j
   0.        +0.j          0.25937061+0.24062939j  0.        +0.j
   0.        +0.j          0.        +0.j          0.        +0.j
   0.        +0.j        ]
 [ 0.        +0.j         -1.74062939-1.74062939j -2.        -2.j
   0.        +0.j          0.        +0.j          0.        +0.j
   0.        +0.j          0.        +0.j          0.        +0.j
   0.        +0.j          0.        +0.j          0.25937061+0.24062939j
   0.        +0.j          0.        +0.j          0.        +0.j
   0.        +0.j        ]
 [ 0.        +0.j         -2.        -2.j         -1.75      -1.75j
   0.        +0.j          0.        +0.j          0.        +0.j
   0.        +0.j          0.5       +0.5j         0.        +0.j
   0.        +0.j          0.        +0.j          0.